## Please don't run / don't click "Run all" the notebook:
At the time of writing of this document, the current core limit is 200 cores per workspace and depending upon number of concurrent users, you may end up with core capacity being exceeded or maximum number of parallel jobs being exceeded error.         



In [3]:
data_path = spark.read.load('abfss://mfgcosmos@#DATA_LAKE_NAME#.dfs.core.windows.net/test/data_696caa9e-0159-4407-9c23-50d6a9033e55_c758459c-7575-47a8-83d1-08f93b356240.json', format='json')
display(data_path)
#data_path.show(100)

## Data Transformation


In [4]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
pd_df = data_path.select("AggregationPeriod","DeviceName","DischargePressureMean",
"PowerMean","SuctionPressureMean").toPandas()
pd_df.groupby(['DeviceName','AggregationPeriod']).mean()
print(pd_df)

AggregationPeriod         ...         SuctionPressureMean
0                  120         ...                   27.770970
1                  120         ...                   27.890951
2                  120         ...                   27.780150
3                  120         ...                   27.861455
4                  120         ...                   27.805039
5                  120         ...                   27.764742
6                  120         ...                   27.869061
7                  120         ...                   27.780556
8                  120         ...                   27.694953
9                  120         ...                   27.847852
10                 120         ...                   27.826671
11                 120         ...                   27.889680
12                 120         ...                   27.901336
13                 120         ...                   27.915035
14                 120         ...                   27.8291

In [5]:
%%pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np
pd_df = data_path.select("AggregationPeriod","DeviceName","DischargePressureMean","PowerMean","SuctionPressureMean").toPandas()
pd_df.groupby(['DeviceName','AggregationPeriod']).sum()

pd_df.head(10)

AggregationPeriod         ...         SuctionPressureMean
0                120         ...                   27.770970
1                120         ...                   27.890951
2                120         ...                   27.780150
3                120         ...                   27.861455
4                120         ...                   27.805039
5                120         ...                   27.764742
6                120         ...                   27.869061
7                120         ...                   27.780556
8                120         ...                   27.694953
9                120         ...                   27.847852

[10 rows x 5 columns]

In [6]:
%%pyspark
df = spark.createDataFrame(pd_df)
df.show(5)

(df
 .coalesce(1)
 .write
 .mode("overwrite")
 .option("header", "true")
 .format("com.databricks.spark.csv")
 .save('abfss://mfgcosmos@#DATA_LAKE_NAME#.dfs.core.windows.net/MFGData'))


+-----------------+----------+---------------------+------------------+-------------------+
|AggregationPeriod|DeviceName|DischargePressureMean|         PowerMean|SuctionPressureMean|
+-----------------+----------+---------------------+------------------+-------------------+
|              120|PQ_4738_CF|    97.63247813206834|1087.8062715758747| 27.770969606738017|
|              120|PQ_4738_CF|    97.62699071731443| 1087.345772417963| 27.890951093380945|
|              120|PQ_4738_CF|    97.67204051745139|1087.5128486783822| 27.780150452295757|
|              120|PQ_4738_CF|    97.36496721547327|1086.3139724110802| 27.861455485000352|
|              120|PQ_4738_CF|    97.54960762091932|1083.7345665778666| 27.805038981769233|
+-----------------+----------+---------------------+------------------+-------------------+
only showing top 5 rows

/opt/spark/python/lib/pyspark.zip/pyspark/sql/session.py:714: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.executio